# Step 0: Install dependencies and set env vars

In [ ]:
%pip install yt_dlp whisper langchain

# Step 1: Download and convert a YouTube video to MP3

In [2]:
# Step 1: Download and convert a YouTube video to MP3
import yt_dlp
import os

youtube_url = "https://www.youtube.com/watch?v=UrcwDOEBzZE"  # Replace with your YouTube URL
output_dir = "downloads"
os.makedirs(output_dir, exist_ok=True)

ydl_opts = {
    'format': 'bestaudio/best',
    'postprocessors': [{
        'key': 'FFmpegExtractAudio',
        'preferredcodec': 'mp3',
        'preferredquality': '192',
    }],
    'outtmpl': os.path.join(output_dir, '%(title)s.%(ext)s')
}

with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    info = ydl.extract_info(youtube_url, download=True)
    mp3_file = os.path.join(output_dir, f"{info['title']}.mp3")

print(f"Downloaded MP3 file: {mp3_file}")


[youtube] Extracting URL: https://www.youtube.com/watch?v=UrcwDOEBzZE
[youtube] UrcwDOEBzZE: Downloading webpage
[youtube] UrcwDOEBzZE: Downloading ios player API JSON
[youtube] UrcwDOEBzZE: Downloading mweb player API JSON
[youtube] UrcwDOEBzZE: Downloading player 6e1dd460
[youtube] UrcwDOEBzZE: Downloading m3u8 information
[info] UrcwDOEBzZE: Downloading 1 format(s): 251


ERROR: unable to download video data: HTTP Error 403: Forbidden


DownloadError: ERROR: unable to download video data: HTTP Error 403: Forbidden

In [ ]:
# Step 2: Transcribe MP3 file using Whisper
import whisper

model_name = "base"
model = whisper.load_model(model_name)
result = model.transcribe(mp3_file)
transcription = result['text']

print("Transcription:", transcription[:500])  # Show first 500 characters

In [ ]:
# Step 3: Convert transcription to vector and store
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_text(transcription)

embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_texts(texts, embeddings)

In [ ]:
# Step 4: Retrieve vectors according to prompt
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI

retriever = vectorstore.as_retriever()
qa_chain = RetrievalQA.from_chain_type(
    llm=OpenAI(),
    retriever=retriever,
    return_source_documents=True
)


In [ ]:
# Step 5: Answer the question
query = "What is the main topic discussed in the video?"
result = qa_chain.run(query)
print("Answer:", result)
